Use Open AI para resumir em texto os videos do Youtube. 

In [10]:
pip install pytubefix ffmpeg-python openai python-dotenv ollama

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from pytubefix import YouTube
from pytubefix.cli import on_progress
import os

url = input()
yt = YouTube(url,  on_progress_callback = on_progress)

ys = yt.streams.get_audio_only()
result = ys.download(mp3=True) # pass the parameter mp3=True to save in .mp3
print("Downloading...")
base, ext = os.path.splitext(result)
new_MP3_file = base + '.mp3'
os.rename(result, new_MP3_file)
print("Downloading... Done")


In [18]:
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY
)

audio_file = open(new_MP3_file, "rb")
TRANSCRIPT_MODEL="whisper-1"
transcript = client.audio.transcriptions.create(
  model=TRANSCRIPT_MODEL, 
  file=audio_file, 
  response_format="text"
)


stream = client.chat.completions.create(
    model=TRANSCRIPT_MODEL,
    messages=[
      {"role": "system", 
      "content": "Você é um assintente que resume audios e videos e formata a resposta em Markdown"},
        {"role": "user", 
        "content": "Por favor, descreva o seguinte video: " + transcript}],
    stream=True,
)
for chunk in stream:
    print(chunk.choices[0].delta.content or "", end="")

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}